<a href="https://colab.research.google.com/github/ditarnnt/Telecom-Predic-Churn/blob/main/ML_IBM_PP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starting steps

Training: Run all

Evaluating: Run dari awal sampai feature selection, lalu run 'Evaluation'

In [ ]:
from google.colab import drive
import pandas as pd #pandas
import numpy as np #numpy
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #encoding data kategorikal
from sklearn.preprocessing import StandardScaler #standardization
from sklearn.model_selection import train_test_split #train test split
from sklearn.ensemble import RandomForestClassifier
import time
import pickle


!pip install ipython-autotime
%load_ext autotime

rand_state = 101 #hyperparameter search argument
niter = 125 #hyperparameter search argument
kfold = 3 #hyperparameter search argument

time: 3.99 ms (started: 2021-05-11 10:49:12 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 5.11 ms (started: 2021-05-11 10:49:12 +00:00)


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/TubesAI_folder/Dataset/IBM_pp.csv')  
print (dataset)

      tenure  MonthlyCharges  ...  PaymentMethod_Mailed check  Churn
0          1           29.85  ...                           0      0
1         34           56.95  ...                           1      0
2          2           53.85  ...                           1      1
3         45           42.30  ...                           0      0
4          2           70.70  ...                           0      1
...      ...             ...  ...                         ...    ...
7027      24           84.80  ...                           1      0
7028      72          103.20  ...                           0      0
7029      11           29.60  ...                           0      0
7030       4           74.40  ...                           1      1
7031      66          105.65  ...                           0      0

[7032 rows x 47 columns]
time: 91.4 ms (started: 2021-05-11 10:49:12 +00:00)


In [ ]:
#Definisi fungsi untuk menyeleksi fitur
def select_features(df,selected_features):
  '''
  Fungsi untuk memilih fitur pada dataset.
  Menggunakan RegEx untuk seleksi fitur yang di-encode secara OneHot Encoding.
  Ketentuan seleksi fitur OneHotEncoding: Fitur harus punya prefix yang sesuai dengan nama fitur.
  Argumen:
  df - dataset yang akan diseleksi fitur-fiturnya
  selected_features - list berisi fitur-fitur yang akan dipilih
  Output:
  df_out - dataset yang berisi fitur-fitur pada selected_features.
  '''
  df_out = pd.DataFrame(index=df.index.tolist())
  cols = []
  for feature in selected_features:
    r = re.compile(feature) # Read Note
    cols = cols + list(filter(r.match,df.columns.tolist()))
  df_out = df[cols]
  return df_out

time: 8.29 ms (started: 2021-05-11 10:49:13 +00:00)


In [ ]:
'''
#List information gain untuk setiap fitur
 0.141647   17 Contract
 0.1095481   1 tenure
 0.0930946  11 OnlineSecurity
 0.0907063  14 TechSupport
 0.0799162  10 InternetService
 0.0673147  12 OnlineBackup
 0.0640887  19 PaymentMethod
 0.0631674  13 DeviceProtection
 0.0624243   2 MonthlyCharges
 0.0543669   3 TotalCharges
 0.0460485  16 StreamingMovies
 0.0458814  15 StreamingTV
 0.0275833  18 PaperlessBilling
 0.0205877   7 Dependents
 0.0164224   6 Partner
 0.0151954   5 SeniorCitizen
 0.0011518   9 MultipleLines
 0.0000998   8 PhoneService
 0.0000527   4 gender
'''

'\n#List information gain untuk setiap fitur\n 0.141647   17 Contract\n 0.1095481   1 tenure\n 0.0930946  11 OnlineSecurity\n 0.0907063  14 TechSupport\n 0.0799162  10 InternetService\n 0.0673147  12 OnlineBackup\n 0.0640887  19 PaymentMethod\n 0.0631674  13 DeviceProtection\n 0.0624243   2 MonthlyCharges\n 0.0543669   3 TotalCharges\n 0.0460485  16 StreamingMovies\n 0.0458814  15 StreamingTV\n 0.0275833  18 PaperlessBilling\n 0.0205877   7 Dependents\n 0.0164224   6 Partner\n 0.0151954   5 SeniorCitizen\n 0.0011518   9 MultipleLines\n 0.0000998   8 PhoneService\n 0.0000527   4 gender\n'

time: 7.11 ms (started: 2021-05-11 10:49:13 +00:00)


In [ ]:
y = dataset['Churn']
X = dataset.drop(['Churn'], axis = 1)

time: 6.6 ms (started: 2021-05-11 10:49:13 +00:00)


In [ ]:
print(f'X: {X.shape}')

X: (7032, 46)
time: 2.83 ms (started: 2021-05-11 10:49:13 +00:00)


# Base model

In [ ]:
#Train dan test split

X_train , X_test, y_train, y_test = train_test_split(X, y , test_size = 0.200 , random_state = rand_state)

time: 10.4 ms (started: 2021-05-11 10:49:13 +00:00)


In [ ]:
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

X_train: (5625, 46)
y_train: (5625,)
X_test: (1407, 46)
y_test: (1407,)
time: 5.92 ms (started: 2021-05-11 10:49:13 +00:00)


In [ ]:
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

time: 2.45 ms (started: 2021-05-11 10:49:13 +00:00)


In [ ]:
classifier = RandomForestClassifier(random_state = rand_state)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

time: 803 ms (started: 2021-05-11 10:49:13 +00:00)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print (classification_report(y_test, y_pred))
print (confusion_matrix(y_test, y_pred))
print (accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1052
           1       0.59      0.49      0.53       355

    accuracy                           0.78      1407
   macro avg       0.71      0.69      0.70      1407
weighted avg       0.77      0.78      0.78      1407

[[931 121]
 [182 173]]
0.7846481876332623
time: 19.7 ms (started: 2021-05-11 10:49:13 +00:00)


# Proposed Model: Feature Selection

In [ ]:
#selected_features = ['Contract', 'tenure', 'OnlineSecurity', 'TechSupport', 'InternetService', 'OnlineBackup', 'PaymentMethod', 
#'DeviceProtection', 'MonthlyCharges', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'PaperlessBilling', 'Dependents',
#'Partner', 'SeniorCitizen','MultipleLines','PhoneService','gender', 'Churn']

selected_features = ['Contract', 'tenure', 'OnlineSecurity', 'TechSupport', 'InternetService', 'OnlineBackup', 'PaymentMethod', 
                     'DeviceProtection', 'MonthlyCharges', 'TotalCharges','StreamingMovies','StreamingTV',
                     'PaperlessBilling','Dependents','Partner','SeniorCitizen','Churn']

time: 1.72 ms (started: 2021-05-11 10:49:14 +00:00)


In [ ]:
df_out = select_features(dataset, selected_features)
y = df_out['Churn']
X = df_out.drop(['Churn'], axis = 1)
X_train , X_test, y_train, y_test = train_test_split(X, y , test_size = 0.200 , random_state = rand_state)

time: 27.5 ms (started: 2021-05-11 10:49:14 +00:00)


# Proposed Model: Hyperparameter Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import cross_val_score

random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 600, 8, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(50, 300, 8, dtype = int))}

clf = RandomForestClassifier()
#model = GridSearchCV(clf, random_search,
#                               cv = 3, verbose= 5, n_jobs = -1)
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = niter, 
                               cv = kfold, verbose= 5, random_state= rand_state, n_jobs = -1)
model.fit(X_train,y_train)

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:  5.5min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

time: 5min 30s (started: 2021-05-11 10:49:14 +00:00)


In [ ]:
predictionforest = model.best_estimator_.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc = accuracy_score(y_test,y_pred)
acc1 = accuracy_score(y_test,predictionforest)

[[966  86]
 [173 182]]
              precision    recall  f1-score   support

           0       0.85      0.92      0.88      1052
           1       0.68      0.51      0.58       355

    accuracy                           0.82      1407
   macro avg       0.76      0.72      0.73      1407
weighted avg       0.81      0.82      0.81      1407

time: 91.1 ms (started: 2021-05-11 10:54:44 +00:00)


In [ ]:
print("Base accuracy:",acc*100,"%")
print("Optimized accuracy:",acc1*100,"%")
print("Increase in accuracy:",(acc1-acc)*100,"%")

Base accuracy: 78.46481876332622 %
Optimized accuracy: 81.59203980099502 %
Increase in accuracy: 3.1272210376687926 %
time: 2.79 ms (started: 2021-05-11 10:54:44 +00:00)


# Save Base & Proposed Model

In [ ]:
#ngesave model

runtime = time.strftime("%d-%m-%Y %H:%M:%S")

base_filename = "/content/drive/MyDrive/TubesAI_folder/Model_IBM/ibm_base_" + runtime + ".pkl"
with open(base_filename, 'wb') as file:
  pickle.dump(classifier, file)

pkl_filename = pkl_filename = "/content/drive/MyDrive/TubesAI_folder/Model_IBM/ibm_model_" + runtime + ".pkl"
with open(pkl_filename, 'wb') as file:
  pickle.dump(model, file)

with open("/content/drive/MyDrive/TubesAI_folder/Model_IBM/log_" + runtime + ".txt",'a') as file:
  L = ["base accuracy = "+str(acc*100)+"%\n",
       "opt accuracy = "+str(acc1*100)+"%\n",
       "Increase in accuracy:"+str((acc1-acc)*100)+"%\n",
       str(model.best_estimator_.get_params())]
  file.writelines(L)

time: 225 ms (started: 2021-05-11 10:54:44 +00:00)


In [ ]:
model.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 600,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 12,
 'min_samples_split': 5,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 264,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

time: 11 ms (started: 2021-05-11 10:54:45 +00:00)


# Evaluation

In [ ]:
infile = open('/content/drive/MyDrive/TubesAI_folder/Model_IBM/ibm_model_'+runtime+'.pkl','rb')
evalmodel = pickle.load(infile)
infile.close()

evalmodel.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 600,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 12,
 'min_samples_split': 5,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 264,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

time: 37.8 ms (started: 2021-05-11 10:54:45 +00:00)


In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

y_eval = evalmodel.best_estimator_.predict(X_test)

acceval = accuracy_score(y_test,y_eval)*100
preciseval = precision_score(y_test,y_eval)*100
recalleval = recall_score(y_test,y_eval)*100
f1eval = f1_score(y_test,y_eval)*100

print("EVALUATION OF PROPOSED MODEL:\n")

print("Summary:")
print(classification_report(y_test,y_eval))

print("Confusion matrix:")
print(str(confusion_matrix(y_test,y_eval))+"\n")

print("Accuracy:")
print(str(acceval)+"%\n")

print("Precision:")
print(str(preciseval)+"%\n")

print("Recall:")
print(str(recalleval)+"%\n")

print("F1 Score:")
print(str(f1eval)+"%\n")

EVALUATION OF PROPOSED MODEL:

Summary:
              precision    recall  f1-score   support

           0       0.85      0.92      0.88      1052
           1       0.68      0.51      0.58       355

    accuracy                           0.82      1407
   macro avg       0.76      0.72      0.73      1407
weighted avg       0.81      0.82      0.81      1407

Confusion matrix:
[[966  86]
 [173 182]]

Accuracy:
81.59203980099502%

Precision:
67.91044776119402%

Recall:
51.267605633802816%

F1 Score:
58.42696629213484%

time: 112 ms (started: 2021-05-11 10:54:45 +00:00)


In [ ]:
accpred = accuracy_score(y_test,y_pred)*100
precispred = precision_score(y_test,y_pred)*100
recallpred = recall_score(y_test,y_pred)*100
f1pred = f1_score(y_test,y_pred)*100

print("EVALUATION OF BASE MODEL:\n")

print("Summary:")
print(classification_report(y_test,y_pred))

print("Confusion matrix:")
print(str(confusion_matrix(y_test,y_pred))+"\n")

print("Accuracy:")
print(str(accpred)+"%\n")

print("Precision:")
print(str(precispred)+"%\n")

print("Recall:")
print(str(recallpred)+"%\n")

print("F1 Score:")
print(str(f1pred)+"%\n")

EVALUATION OF BASE MODEL:

Summary:
              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1052
           1       0.59      0.49      0.53       355

    accuracy                           0.78      1407
   macro avg       0.71      0.69      0.70      1407
weighted avg       0.77      0.78      0.78      1407

Confusion matrix:
[[931 121]
 [182 173]]

Accuracy:
78.46481876332622%

Precision:
58.843537414965986%

Recall:
48.732394366197184%

F1 Score:
53.312788906009246%

time: 34.9 ms (started: 2021-05-11 10:54:45 +00:00)
